In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
!pip install --upgrade tensorflow==1.15

Requirement already up-to-date: tensorflow==1.15 in /usr/local/lib/python3.7/dist-packages (1.15.0)


In [ ]:
layer = {
    'input': 1,
    'output': 1
}

# VARIABLE PEMBANTU
batch_size = 2
time_step = 12
epoch = 1000

In [ ]:
def load_dataset():
    dataset = pd.read_csv('Alcohol_Sales.csv', index_col='DATE')
    dataset.index = pd.to_datetime(dataset.index)
    return dataset

In [ ]:
def get_batch(dataset, time_step, batch_size):
    input_batch = np.zeros(shape=(batch_size, time_step, layer['input']))
    output_batch = np.zeros(shape=(batch_size, time_step, layer['output']))

    for i in range(batch_size)):
        point = np.random.randint(0, len(dataset) - time_step)
        input_batch += dataset[point : point + time_step]
        output_batch += dataset[point + 1 : point + time_step + 1]

    return input_batch, output_batch

In [ ]:
dataset = load_dataset()

In [ ]:
dataset

,S4248SM144NCEN
DATE,
1992-01-01,3459
1992-02-01,3458
1992-03-01,4002
1992-04-01,4564
1992-05-01,4221
...,...
2018-09-01,12396
2018-10-01,13914
2018-11-01,14174


In [ ]:
# TRAIN TEST SPLIT
split = int(len(dataset) * 0.8)
train_dataset = dataset[:split]
test_dataset = dataset[split:]

In [ ]:
# NORMALISASI
scaler = MinMaxScaler().fit(train_dataset)
normalize_train_dataset = scaler.transform(train_dataset)

In [ ]:
normalize_train_dataset

array([[0.04658757],
       [0.04647872],
       [0.10569283],
       [0.16686622],
       [0.12953086],
       [0.16305649],
       [0.15619898],
       [0.1203875 ],
       [0.11919016],
       [0.13366714],
       [0.131599  ],
       [0.20735822],
       [0.        ],
       [0.02503538],
       [0.12289104],
       [0.14651137],
       [0.13889191],
       [0.18123435],
       [0.15532818],
       [0.15521933],
       [0.14509633],
       [0.1311636 ],
       [0.15989986],
       [0.2114945 ],
       [0.00478938],
       [0.03766191],
       [0.15369544],
       [0.13388484],
       [0.15554588],
       [0.21269185],
       [0.14074235],
       [0.18373789],
       [0.14618483],
       [0.13246979],
       [0.19081311],
       [0.21116795],
       [0.03689997],
       [0.04930881],
       [0.13475563],
       [0.10482203],
       [0.18330249],
       [0.2114945 ],
       [0.12974856],
       [0.21432459],
       [0.13105475],
       [0.16577773],
       [0.19810602],
       [0.195

In [ ]:
# BUILDING MODEL
cell = tf.nn.rnn_cell.BasicRNNCell(10, activation = tf.nn.relu)
cell = tf.contrib.rnn.OutputProjectionWrapper(cell, layer['output'], activation = tf.nn.relu)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
cell

In [ ]:
input_feature = tf.placeholder(tf.float32, [None, time_step, layer['input']])
input_target = tf.placeholder(tf.float32, [None, time_step, layer['output']])

In [ ]:
output, _ = tf.nn.dynamic_rnn(cell, input_feature, dtype = tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# MSE
loss = tf.reduce_mean(0.5 * (input_target - output) ** 2)
train = tf.train.AdamOptimizer(0.0001).minimize(loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(1, epoch + 1):
        input_batch, output_batch = get_batch(normalize_train_dataset, time_step, batch_size)
        print(input_batch.shape)
        feed = { 
            input_feature: input_batch,
            input_target: output_batch
        }
        sess.run(train, feed_dict = feed)

        if i % 200 == 0:
            print("Iteration: {}, Loss: {}" .format(i, sess.run(loss, feed_dict = feed)))

    saver.save(sess, './rnn-model.ckpt') # checkpoint

In [ ]:
with tf.Session() as sess:
    seed_data = list(normalize_train_dataset)
    saver.restore(sess, './rnn-model.ckpt')

    for i in range(len(test_dataset)):
        input_batch = np.array(seed_data[-time_step:]).reshape(1, time_step, layer['input'])
        print(input_batch.shape)
        feed = {
            input_feature: input_batch
        }
        predict = sess.run(output, feed_dict = feed)
        seed_data.append(predict[0, -1, 0])

    predict_result = scaler.inverse_transform(np.array(seed_data[-len(test_dataset):]).reshape(len(test_dataset), 1))
    test_dataset['Prediction'] = predict_result
    test_dataset.plot()
    plt.show()